# ENV Setup


In [1]:
! pip install torch adam-atan2 einops tqdm coolname pydantic argdantic wandb omegaconf hydra-core huggingface_hub packaging ninja wheel setuptools setuptools-scm pydantic-core numba triton

  Preparing metadata (setup.py) ... done
  Using cached setuptools_scm-9.2.2-py3-none-any.whl.metadata (7.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 19.9 MB/s eta 0:00:00
Using cached setuptools_scm-9.2.2-py3-none-any.whl (62 kB)
  Created wheel for adam-atan2: filename=adam_atan2-0.0.3-cp312-cp312-linux_x86_64.whl size=208277 sha256=73bb9413d9dfd4740818df2582587cd3868e426f9e6f3d60699adb570c612e8d
  Stored in directory: /root/.cache/pip/wheels/45/32/31/044d4e2d7dc14d2820c0232d9f41b3ae8cb3d8451451a80dbf
Successfully built adam-atan2


## Weights & Biases

In [4]:
import os

from google.colab import userdata
WANDB_key = userdata.get('WANDB')

# ═══════════════════════════════════════════════════════════════════════════
# Cell: Wandb 自动登录
# ═══════════════════════════════════════════════════════════════════════════

print("=" * 70)
print("📊 配置 Weights & Biases")
print("=" * 70)
import wandb
wandb.login(key=WANDB_key)  # login if you want the logger to sync results to your Weights & Biases (https://wandb.ai/)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jarviszhang (jarviszhang-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


📊 配置 Weights & Biases


True

## Set Seed


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from einops import rearrange, repeat
import numpy as np
import json
import os
import random
import matplotlib.pyplot as plt

# 设置随机种子，保证复现结果一致
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Build Sudoku Dataset

In [6]:
# Colab 独立代码 - 下载并构建 Sudoku 数据集

import os
import csv
import json
import numpy as np
from tqdm import tqdm

# 安装并导入 huggingface_hub
# !pip install huggingface_hub
from huggingface_hub import hf_hub_download

# ============ 配置 ============
SOURCE_REPO = "sapientinc/sudoku-extreme"
SUBSAMPLE_SIZE = 1000  # 训练集采样数量，None 表示全部
NUM_AUG = 1         # 数据增强次数

OUTPUT_DIR = f"data/sudoku-extreme-${SUBSAMPLE_SIZE}-aug-${NUM_AUG}"

# ============ 数据增强函数 ============
def shuffle_sudoku(board: np.ndarray, solution: np.ndarray):
    """对数独进行等价变换（保持有效性）"""
    # 数字映射：随机置换 1-9
    digit_map = np.pad(np.random.permutation(np.arange(1, 10)), (1, 0))

    # 随机转置
    transpose_flag = np.random.rand() < 0.5

    # 行置换：先打乱 3 个 band，再打乱每个 band 内的 3 行
    bands = np.random.permutation(3)
    row_perm = np.concatenate([b * 3 + np.random.permutation(3) for b in bands])

    # 列置换：同理
    stacks = np.random.permutation(3)
    col_perm = np.concatenate([s * 3 + np.random.permutation(3) for s in stacks])

    # 构建 81->81 的位置映射
    mapping = np.array([row_perm[i // 9] * 9 + col_perm[i % 9] for i in range(81)])

    def apply_transformation(x: np.ndarray) -> np.ndarray:
        if transpose_flag:
            x = x.T
        new_board = x.flatten()[mapping].reshape(9, 9).copy()
        return digit_map[new_board]

    return apply_transformation(board), apply_transformation(solution)

# ============ 处理单个子集 ============
def convert_subset(set_name: str):
    print(f"Processing {set_name} set...")

    # 从 HuggingFace 下载 CSV
    csv_path = hf_hub_download(SOURCE_REPO, f"{set_name}.csv", repo_type="dataset")

    # 读取 CSV
    inputs, labels = [], []
    with open(csv_path, newline="") as f:
        reader = csv.reader(f)
        next(reader)  # 跳过 header
        for source, q, a, rating in reader:
            inputs.append(np.frombuffer(q.replace('.', '0').encode(), dtype=np.uint8).reshape(9, 9) - ord('0'))
            labels.append(np.frombuffer(a.encode(), dtype=np.uint8).reshape(9, 9) - ord('0'))

    print(f"  Loaded {len(inputs)} puzzles")

    # 训练集采样
    if set_name == "train" and SUBSAMPLE_SIZE is not None and SUBSAMPLE_SIZE < len(inputs):
        indices = np.random.choice(len(inputs), size=SUBSAMPLE_SIZE, replace=False)
        inputs = [inputs[i] for i in indices]
        labels = [labels[i] for i in indices]
        print(f"  Subsampled to {len(inputs)} puzzles")

    # 数据增强（仅训练集）
    num_augments = NUM_AUG if set_name == "train" else 0

    # 构建结果
    results = {k: [] for k in ["inputs", "labels", "puzzle_identifiers", "puzzle_indices", "group_indices"]}
    puzzle_id = 0
    example_id = 0

    results["puzzle_indices"].append(0)
    results["group_indices"].append(0)

    for orig_inp, orig_out in tqdm(zip(inputs, labels), total=len(inputs)):
        for aug_idx in range(1 + num_augments):
            if aug_idx == 0:
                inp, out = orig_inp, orig_out
            else:
                inp, out = shuffle_sudoku(orig_inp, orig_out)

            results["inputs"].append(inp)
            results["labels"].append(out)
            example_id += 1
            puzzle_id += 1

            results["puzzle_indices"].append(example_id)
            results["puzzle_identifiers"].append(0)

        results["group_indices"].append(puzzle_id)

    # 转换为 NumPy 数组
    def seq_to_numpy(seq):
        arr = np.concatenate(seq).reshape(len(seq), -1)
        return arr + 1  # 偏移 +1，0 留给 PAD

    results = {
        "inputs": seq_to_numpy(results["inputs"]),
        "labels": seq_to_numpy(results["labels"]),
        "group_indices": np.array(results["group_indices"], dtype=np.int32),
        "puzzle_indices": np.array(results["puzzle_indices"], dtype=np.int32),
        "puzzle_identifiers": np.array(results["puzzle_identifiers"], dtype=np.int32),
    }

    # 元数据
    metadata = {
        "seq_len": 81,
        "vocab_size": 11,  # PAD + 0-9
        "pad_id": 0,
        "ignore_label_id": 0,
        "blank_identifier_id": 0,
        "num_puzzle_identifiers": 1,
        "total_groups": len(results["group_indices"]) - 1,
        "mean_puzzle_examples": 1,
        "total_puzzles": len(results["group_indices"]) - 1,
        "sets": ["all"]
    }

    # 保存
    save_dir = os.path.join(OUTPUT_DIR, set_name)
    os.makedirs(save_dir, exist_ok=True)

    with open(os.path.join(save_dir, "dataset.json"), "w") as f:
        json.dump(metadata, f)

    for k, v in results.items():
        np.save(os.path.join(save_dir, f"all__{k}.npy"), v)

    print(f"  Saved to {save_dir}")

# ============ 主程序 ============
if __name__ == "__main__":
    convert_subset("train")
    convert_subset("test")

    # 保存 identifiers.json
    with open(os.path.join(OUTPUT_DIR, "identifiers.json"), "w") as f:
        json.dump(["<blank>"], f)

    print(f"\nDone! Dataset saved to {OUTPUT_DIR}")

Processing train set...


train.csv:   0%|          | 0.00/719M [00:00<?, ?B/s]

  Loaded 3831994 puzzles
  Subsampled to 1000 puzzles


100%|██████████| 1000/1000 [00:00<00:00, 7171.23it/s]


  Saved to data/sudoku-extreme-$1000-aug-$1/train
Processing test set...


test.csv:   0%|          | 0.00/79.4M [00:00<?, ?B/s]

  Loaded 422786 puzzles


100%|██████████| 422786/422786 [00:00<00:00, 1867518.67it/s]


  Saved to data/sudoku-extreme-$1000-aug-$1/test

Done! Dataset saved to data/sudoku-extreme-$1000-aug-$1


## PuzzleDataset

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
import os

class PuzzleDataset(Dataset):
    def __init__(self, data_dir: str, split: str = "train"):
        """
        Args:
            data_dir: e.g., "data/sudoku-extreme-$1000-aug-$1"
            split: "train" or "test"
        """
        split_dir = os.path.join(data_dir, split)

        # 加载元数据
        with open(os.path.join(split_dir, "dataset.json")) as f:
            self.metadata = json.load(f)

        # 加载数据
        self.inputs = np.load(os.path.join(split_dir, "all__inputs.npy"))
        self.labels = np.load(os.path.join(split_dir, "all__labels.npy"))
        self.puzzle_identifiers = np.load(os.path.join(split_dir, "all__puzzle_identifiers.npy"))

        print(f"Loaded {len(self.inputs)} samples from {split_dir}")

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "inputs": torch.tensor(self.inputs[idx], dtype=torch.long),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
            "puzzle_id": torch.tensor(self.puzzle_identifiers[idx], dtype=torch.long),
        }

# 测试
if __name__ == "__main__":
    dataset = PuzzleDataset(f"data/sudoku-extreme-${SUBSAMPLE_SIZE}-aug-${NUM_AUG}", "train")
    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    batch = next(iter(loader))
    print(f"inputs shape: {batch['inputs'].shape}")   # [32, 81]
    print(f"labels shape: {batch['labels'].shape}")   # [32, 81]

Loaded 2000 samples from data/sudoku-extreme-$1000-aug-$1/train
inputs shape: torch.Size([32, 81])
labels shape: torch.Size([32, 81])


# TRM 模型核心组件

## TRM

In [10]:
"""
TinyRecursiveModel (TRM) - 完整复现
与原项目 models/recursive_reasoning/trm.py 逻辑一致
"""

import math
from typing import Tuple, Dict, Optional
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
import einops


# ============ 工具函数 ============

def trunc_normal_init_(tensor: torch.Tensor, std: float = 1.0, lower: float = -2.0, upper: float = 2.0):
    """截断正态初始化 (与 JAX 一致)"""
    with torch.no_grad():
        if std == 0:
            tensor.zero_()
        else:
            sqrt2 = math.sqrt(2)
            a = math.erf(lower / sqrt2)
            b = math.erf(upper / sqrt2)
            z = (b - a) / 2
            c = (2 * math.pi) ** -0.5
            pdf_u = c * math.exp(-0.5 * lower ** 2)
            pdf_l = c * math.exp(-0.5 * upper ** 2)
            comp_std = std / math.sqrt(1 - (upper * pdf_u - lower * pdf_l) / z - ((pdf_u - pdf_l) / z) ** 2)
            tensor.uniform_(a, b)
            tensor.erfinv_()
            tensor.mul_(sqrt2 * comp_std)
            tensor.clip_(lower * comp_std, upper * comp_std)
    return tensor


def rms_norm(hidden_states: torch.Tensor, variance_epsilon: float = 1e-5) -> torch.Tensor:
    """RMS Normalization"""
    input_dtype = hidden_states.dtype
    hidden_states = hidden_states.to(torch.float32)
    variance = hidden_states.square().mean(-1, keepdim=True)
    hidden_states = hidden_states * torch.rsqrt(variance + variance_epsilon)
    return hidden_states.to(input_dtype)


def _find_multiple(a, b):
    """向上取整到 b 的倍数"""
    return (-(a // -b)) * b


# ============ 基础层 ============

class CastedLinear(nn.Module):
    """带类型转换的线性层"""
    def __init__(self, in_features: int, out_features: int, bias: bool = False):
        super().__init__()
        self.weight = nn.Parameter(
            trunc_normal_init_(torch.empty((out_features, in_features)), std=1.0 / (in_features ** 0.5))
        )
        self.bias = nn.Parameter(torch.zeros(out_features)) if bias else None

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        weight = self.weight.to(x.dtype)
        bias = self.bias.to(x.dtype) if self.bias is not None else None
        return F.linear(x, weight, bias)


class CastedEmbedding(nn.Module):
    """带类型转换的 Embedding"""
    def __init__(self, num_embeddings: int, embedding_dim: int, init_std: float, cast_to: torch.dtype):
        super().__init__()
        self.cast_to = cast_to
        self.embedding_weight = nn.Parameter(
            trunc_normal_init_(torch.empty((num_embeddings, embedding_dim)), std=init_std)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return F.embedding(x, self.embedding_weight.to(self.cast_to))


class SwiGLU(nn.Module):
    """SwiGLU FFN"""
    def __init__(self, hidden_size: int, expansion: float = 4.0):
        super().__init__()
        inter = _find_multiple(round(expansion * hidden_size * 2 / 3), 256)
        self.gate_up_proj = CastedLinear(hidden_size, inter * 2, bias=False)
        self.down_proj = CastedLinear(inter, hidden_size, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        gate, up = self.gate_up_proj(x).chunk(2, dim=-1)
        return self.down_proj(F.silu(gate) * up)


class RotaryEmbedding(nn.Module):
    """Rotary Position Embedding (RoPE)"""
    def __init__(self, dim: int, max_position_embeddings: int, base: float = 10000.0):
        super().__init__()
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2, dtype=torch.float32) / dim))
        t = torch.arange(max_position_embeddings, dtype=torch.float32)
        freqs = torch.outer(t, inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos_cached", emb.cos(), persistent=False)
        self.register_buffer("sin_cached", emb.sin(), persistent=False)

    def forward(self):
        return self.cos_cached, self.sin_cached


def rotate_half(x: torch.Tensor):
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q: torch.Tensor, k: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor):
    orig_dtype = q.dtype
    q, k = q.to(cos.dtype), k.to(cos.dtype)
    q_embed = (q * cos.unsqueeze(-2)) + (rotate_half(q) * sin.unsqueeze(-2))
    k_embed = (k * cos.unsqueeze(-2)) + (rotate_half(k) * sin.unsqueeze(-2))
    return q_embed.to(orig_dtype), k_embed.to(orig_dtype)


class Attention(nn.Module):
    """Multi-Head Self-Attention"""
    def __init__(self, hidden_size: int, num_heads: int, causal: bool = False):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads

        self.qkv_proj = CastedLinear(hidden_size, 3 * hidden_size, bias=False)
        self.o_proj = CastedLinear(hidden_size, hidden_size, bias=False)
        self.causal = causal

    def forward(self, hidden_states: torch.Tensor, cos_sin: Optional[Tuple[torch.Tensor, torch.Tensor]] = None) -> torch.Tensor:
        B, L, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states).view(B, L, 3, self.num_heads, self.head_dim)
        q, k, v = qkv[:, :, 0], qkv[:, :, 1], qkv[:, :, 2]

        if cos_sin is not None:
            cos, sin = cos_sin
            q, k = apply_rotary_pos_emb(q, k, cos, sin)

        # [B, L, H, D] -> [B, H, L, D]
        q, k, v = map(lambda t: t.transpose(1, 2), (q, k, v))
        attn_output = F.scaled_dot_product_attention(q, k, v, is_causal=self.causal)
        attn_output = attn_output.transpose(1, 2).reshape(B, L, self.hidden_size)
        return self.o_proj(attn_output)


# ============ TRM Block ============

class TRMBlock(nn.Module):
    """单个 Transformer Block (Post-Norm)"""
    def __init__(self, hidden_size: int, num_heads: int, expansion: float = 4.0, rms_norm_eps: float = 1e-5):
        super().__init__()
        self.self_attn = Attention(hidden_size, num_heads, causal=False)
        self.mlp = SwiGLU(hidden_size, expansion)
        self.norm_eps = rms_norm_eps

    def forward(self, hidden_states: torch.Tensor, cos_sin=None) -> torch.Tensor:
        # Post-Norm: x = norm(x + attn(x))
        hidden_states = rms_norm(hidden_states + self.self_attn(hidden_states, cos_sin), self.norm_eps)
        hidden_states = rms_norm(hidden_states + self.mlp(hidden_states), self.norm_eps)
        return hidden_states


class ReasoningModule(nn.Module):
    """推理模块：多层 TRMBlock + input injection"""
    def __init__(self, num_layers: int, hidden_size: int, num_heads: int, expansion: float = 4.0):
        super().__init__()
        self.layers = nn.ModuleList([
            TRMBlock(hidden_size, num_heads, expansion) for _ in range(num_layers)
        ])

    def forward(self, hidden_states: torch.Tensor, input_injection: torch.Tensor, cos_sin=None) -> torch.Tensor:
        hidden_states = hidden_states + input_injection  # 关键：注入输入
        for layer in self.layers:
            hidden_states = layer(hidden_states, cos_sin)
        return hidden_states


# ============ Carry (状态) ============

@dataclass
class TRMInnerCarry:
    """内部递归状态"""
    z_H: torch.Tensor  # High-level latent
    z_L: torch.Tensor  # Low-level latent


@dataclass
class TRMCarry:
    """完整 ACT 状态"""
    inner_carry: TRMInnerCarry
    steps: torch.Tensor
    halted: torch.Tensor
    current_data: Dict[str, torch.Tensor]


# ============ TRM Inner Model ============

class TRMInner(nn.Module):
    """TRM 核心模型（不含 ACT wrapper）"""
    def __init__(
        self,
        vocab_size: int,
        seq_len: int,
        hidden_size: int = 512,
        num_heads: int = 8,
        expansion: float = 4.0,
        L_layers: int = 2,
        H_cycles: int = 3,
        L_cycles: int = 6,
        pos_encodings: str = "rope",
        rope_theta: float = 10000.0,
        rms_norm_eps: float = 1e-5,
        forward_dtype: str = "bfloat16",
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.H_cycles = H_cycles
        self.L_cycles = L_cycles
        self.forward_dtype = getattr(torch, forward_dtype)
        self.rms_norm_eps = rms_norm_eps

        # Embedding
        self.embed_scale = math.sqrt(hidden_size)
        embed_init_std = 1.0 / self.embed_scale
        self.embed_tokens = CastedEmbedding(vocab_size, hidden_size, embed_init_std, self.forward_dtype)

        # Output heads
        self.lm_head = CastedLinear(hidden_size, vocab_size, bias=False)
        self.q_head = CastedLinear(hidden_size, 2, bias=True)
        # Q head 特殊初始化
        with torch.no_grad():
            self.q_head.weight.zero_()
            self.q_head.bias.fill_(-5)

        # Position encoding
        self.pos_encodings = pos_encodings
        if pos_encodings == "rope":
            self.rotary_emb = RotaryEmbedding(hidden_size // num_heads, seq_len, rope_theta)

        # Reasoning layers (L_level)
        self.L_level = ReasoningModule(L_layers, hidden_size, num_heads, expansion)

        # Initial states (可学习)
        self.H_init = nn.Parameter(trunc_normal_init_(torch.empty(hidden_size, dtype=self.forward_dtype), std=1))
        self.L_init = nn.Parameter(trunc_normal_init_(torch.empty(hidden_size, dtype=self.forward_dtype), std=1))

    def _input_embeddings(self, inputs: torch.Tensor) -> torch.Tensor:
        embedding = self.embed_tokens(inputs.to(torch.int32))
        return self.embed_scale * embedding

    def empty_carry(self, batch_size: int) -> TRMInnerCarry:
        return TRMInnerCarry(
            z_H=torch.empty(batch_size, self.seq_len, self.hidden_size, dtype=self.forward_dtype, device=self.H_init.device),
            z_L=torch.empty(batch_size, self.seq_len, self.hidden_size, dtype=self.forward_dtype, device=self.L_init.device),
        )

    def reset_carry(self, reset_flag: torch.Tensor, carry: TRMInnerCarry) -> TRMInnerCarry:
        """重置已 halt 的序列的 carry"""
        return TRMInnerCarry(
            z_H=torch.where(reset_flag.view(-1, 1, 1), self.H_init, carry.z_H),
            z_L=torch.where(reset_flag.view(-1, 1, 1), self.L_init, carry.z_L),
        )

    def forward(self, carry: TRMInnerCarry, batch: Dict[str, torch.Tensor]):
        cos_sin = self.rotary_emb() if hasattr(self, "rotary_emb") else None

        # Input encoding
        input_embeddings = self._input_embeddings(batch["inputs"])

        z_H, z_L = carry.z_H, carry.z_L

        # ========== 核心递归逻辑 ==========
        # H_cycles - 1 次不计算梯度（节省显存）
        with torch.no_grad():
            for _ in range(self.H_cycles - 1):
                for _ in range(self.L_cycles):
                    z_L = self.L_level(z_L, z_H + input_embeddings, cos_sin)
                z_H = self.L_level(z_H, z_L, cos_sin)

        # 最后 1 次计算梯度
        for _ in range(self.L_cycles):
            z_L = self.L_level(z_L, z_H + input_embeddings, cos_sin)
        z_H = self.L_level(z_H, z_L, cos_sin)
        # ==================================

        # Output
        new_carry = TRMInnerCarry(z_H=z_H.detach(), z_L=z_L.detach())
        logits = self.lm_head(z_H)
        q_logits = self.q_head(z_H[:, 0]).to(torch.float32)  # 用第一个位置预测 halt

        return new_carry, logits, (q_logits[..., 0], q_logits[..., 1])


# ============ TRM with ACT ============

class TRM(nn.Module):
    """完整 TRM 模型（含 ACT wrapper）"""
    def __init__(
        self,
        vocab_size: int,
        seq_len: int,
        hidden_size: int = 512,
        num_heads: int = 8,
        expansion: float = 4.0,
        L_layers: int = 2,
        H_cycles: int = 3,
        L_cycles: int = 6,
        halt_max_steps: int = 16,
        halt_exploration_prob: float = 0.1,
        **kwargs
    ):
        super().__init__()
        self.inner = TRMInner(
            vocab_size=vocab_size,
            seq_len=seq_len,
            hidden_size=hidden_size,
            num_heads=num_heads,
            expansion=expansion,
            L_layers=L_layers,
            H_cycles=H_cycles,
            L_cycles=L_cycles,
            **kwargs
        )
        self.halt_max_steps = halt_max_steps
        self.halt_exploration_prob = halt_exploration_prob

    def initial_carry(self, batch: Dict[str, torch.Tensor]) -> TRMCarry:
        batch_size = batch["inputs"].shape[0]
        device = batch["inputs"].device
        return TRMCarry(
            inner_carry=self.inner.empty_carry(batch_size),
            steps=torch.zeros(batch_size, dtype=torch.int32, device=device),
            halted=torch.ones(batch_size, dtype=torch.bool, device=device),  # 初始为 halted
            current_data={k: torch.empty_like(v) for k, v in batch.items()}
        )

    def forward(self, carry: TRMCarry, batch: Dict[str, torch.Tensor]):
        # 重置已 halt 的序列
        new_inner_carry = self.inner.reset_carry(carry.halted, carry.inner_carry)
        new_steps = torch.where(carry.halted, 0, carry.steps)
        new_current_data = {
            k: torch.where(carry.halted.view((-1,) + (1,) * (batch[k].ndim - 1)), batch[k], v)
            for k, v in carry.current_data.items()
        }

        # Forward
        new_inner_carry, logits, (q_halt_logits, q_continue_logits) = self.inner(new_inner_carry, new_current_data)

        outputs = {
            "logits": logits,
            "q_halt_logits": q_halt_logits,
            "q_continue_logits": q_continue_logits,
        }

        # ACT halting logic
        with torch.no_grad():
            new_steps = new_steps + 1
            is_last_step = new_steps >= self.halt_max_steps
            halted = is_last_step

            if self.training and self.halt_max_steps > 1:
                halted = halted | (q_halt_logits > 0)
                # Exploration
                min_halt_steps = (torch.rand_like(q_halt_logits) < self.halt_exploration_prob) * \
                                 torch.randint_like(new_steps, low=2, high=self.halt_max_steps + 1)
                halted = halted & (new_steps >= min_halt_steps)

        new_carry = TRMCarry(new_inner_carry, new_steps, halted, new_current_data)
        return new_carry, outputs


# ============ 测试 ============
if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = TRM(
        vocab_size=11,  # PAD + 0-9
        seq_len=81,     # 9x9 sudoku
        hidden_size=512,
        num_heads=8,
        L_layers=2,
        H_cycles=3,
        L_cycles=6,
    ).to(device)

    # 模拟输入
    batch = {
        "inputs": torch.randint(1, 11, (4, 81), device=device),
        "labels": torch.randint(1, 11, (4, 81), device=device),
    }

    carry = model.initial_carry(batch)
    carry, outputs = model(carry, batch)

    print(f"logits shape: {outputs['logits'].shape}")  # [4, 81, 11]
    print(f"q_halt_logits shape: {outputs['q_halt_logits'].shape}")  # [4]
    print(f"Model params: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")

logits shape: torch.Size([4, 81, 11])
q_halt_logits shape: torch.Size([4])
Model params: 6.83M


## Loss

In [ ]:
"""
Loss 函数 + 完整训练循环
与原项目 models/losses.py 和 pretrain.py 逻辑一致
"""

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim import AdamW
import numpy as np
import json
import os
from tqdm import tqdm
import wandb  # 新增


IGNORE_LABEL_ID = 0  # PAD token


# ============ Stablemax Cross Entropy (原项目使用) ============

def s(x, epsilon=1e-30):
    """Stablemax 的 s 函数"""
    return torch.where(x < 0, 1 / (1 - x + epsilon), x + 1)


def log_stablemax(x, dim=-1):
    """Stablemax log-probabilities"""
    s_x = s(x)
    return torch.log(s_x / torch.sum(s_x, dim=dim, keepdim=True))


def stablemax_cross_entropy(logits, labels, ignore_index=0):
    """
    Stablemax Cross Entropy Loss
    比 softmax 更稳定，避免数值溢出
    """
    logprobs = log_stablemax(logits.to(torch.float64), dim=-1)
    valid_mask = (labels != ignore_index)
    transformed_labels = torch.where(valid_mask, labels, 0)
    prediction_logprobs = torch.gather(
        logprobs,
        index=transformed_labels.to(torch.long).unsqueeze(-1),
        dim=-1
    ).squeeze(-1)
    return -torch.where(valid_mask, prediction_logprobs, 0)


# ============ ACT Loss Head ============

def compute_loss(outputs, labels, ignore_index=0):
    """
    计算完整 loss：LM loss + Q halt loss
    """
    logits = outputs["logits"]
    q_halt_logits = outputs["q_halt_logits"]

    # Mask for valid tokens
    mask = (labels != ignore_index)
    loss_counts = mask.sum(-1)
    loss_divisor = loss_counts.clamp_min(1).unsqueeze(-1)

    # LM Loss (stablemax cross entropy)
    lm_loss = (stablemax_cross_entropy(logits, labels, ignore_index) / loss_divisor).sum()

    # Correctness for Q halt target
    with torch.no_grad():
        preds = torch.argmax(logits, dim=-1)
        is_correct = mask & (preds == labels)
        seq_is_correct = (is_correct.sum(-1) == loss_counts).float()

    # Q Halt Loss (binary cross entropy)
    q_halt_loss = F.binary_cross_entropy_with_logits(
        q_halt_logits,
        seq_is_correct,
        reduction="sum"
    )

    # Total loss
    total_loss = lm_loss + 0.5 * q_halt_loss

    # Metrics
    metrics = {
        "lm_loss": lm_loss.item(),
        "q_halt_loss": q_halt_loss.item(),
        "token_accuracy": (is_correct.sum() / mask.sum()).item(),      # 改名更清晰
        "exact_accuracy": seq_is_correct.mean().item(),
    }

    return total_loss, metrics


# ============ Dataset ============

class PuzzleDataset(Dataset):
    def __init__(self, data_dir: str, split: str = "train"):
        split_dir = os.path.join(data_dir, split)

        with open(os.path.join(split_dir, "dataset.json")) as f:
            self.metadata = json.load(f)

        self.inputs = np.load(os.path.join(split_dir, "all__inputs.npy"))
        self.labels = np.load(os.path.join(split_dir, "all__labels.npy"))

        print(f"Loaded {len(self.inputs)} samples from {split_dir}")

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "inputs": torch.tensor(self.inputs[idx], dtype=torch.long),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }


# ============ Learning Rate Scheduler ============

def cosine_schedule_with_warmup(current_step, num_warmup_steps, num_training_steps, base_lr, min_ratio=0.0):
    """Cosine learning rate schedule with warmup"""
    if current_step < num_warmup_steps:
        return base_lr * float(current_step) / float(max(1, num_warmup_steps))

    progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
    return base_lr * (min_ratio + max(0.0, (1 - min_ratio) * 0.5 * (1.0 + np.cos(np.pi * progress))))


# ============ Training Loop ============

def train(
    model,
    train_loader,
    test_loader=None,
    epochs=1000,
    lr=1e-4,
    weight_decay=1.0,
    warmup_steps=200,
    eval_interval=100,
    device="cuda",
):
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay, betas=(0.9, 0.95))

    total_steps = epochs * len(train_loader)
    global_step = 0

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        epoch_tok_acc = 0
        epoch_exact_acc = 0
        num_batches = 0

        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for batch in pbar:
            # Move to device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Learning rate schedule
            current_lr = cosine_schedule_with_warmup(
                global_step, warmup_steps, total_steps, lr
            )
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_lr

            # Forward with ACT loop
            carry = model.initial_carry(batch)
            total_loss = 0

            # ACT: 多次 forward 直到全部 halt
            while True:
                carry, outputs = model(carry, batch)

                # 只对 halted 的样本计算 loss
                if carry.halted.any():
                    halted_mask = carry.halted
                    halted_outputs = {
                        "logits": outputs["logits"][halted_mask],
                        "q_halt_logits": outputs["q_halt_logits"][halted_mask],
                    }
                    halted_labels = carry.current_data["labels"][halted_mask]

                    loss, metrics = compute_loss(halted_outputs, halted_labels)
                    total_loss = total_loss + loss

                if carry.halted.all():
                    break

            # Backward
            optimizer.zero_grad()
            (total_loss / len(batch["inputs"])).backward()
            optimizer.step()

            epoch_loss += metrics["lm_loss"]
            epoch_tok_acc += metrics["token_accuracy"]
            epoch_exact_acc += metrics["exact_accuracy"]
            num_batches += 1
            global_step += 1

            # 更新进度条：增加 token_accuracy
            pbar.set_postfix({
                "loss": f"{metrics['lm_loss']:.4f}",
                "tok_acc": f"{metrics['token_accuracy']:.2%}",
                "exact_acc": f"{metrics['exact_accuracy']:.2%}",
                "lr": f"{current_lr:.2e}"
            })

            # W&B 记录每个 step
            wandb.log({
                "train/lm_loss": metrics["lm_loss"],
                "train/q_halt_loss": metrics["q_halt_loss"],
                "train/token_accuracy": metrics["token_accuracy"],
                "train/exact_accuracy": metrics["exact_accuracy"],
                "train/lr": current_lr,
            }, step=global_step)

        # Epoch 结束后记录平均值
        wandb.log({
            "epoch": epoch + 1,
            "train/epoch_lm_loss": epoch_loss / num_batches,
            "train/epoch_token_accuracy": epoch_tok_acc / num_batches,
            "train/epoch_exact_accuracy": epoch_exact_acc / num_batches,
        }, step=global_step)

        # Evaluation
        if test_loader is not None and (epoch + 1) % eval_interval == 0:
            eval_metrics = evaluate(model, test_loader, device)
            print(f"\n[Eval] Epoch {epoch+1}: tok_acc={eval_metrics['token_accuracy']:.2%}, exact_acc={eval_metrics['exact_accuracy']:.2%}")

            # W&B 记录 eval
            wandb.log({
                "eval/token_accuracy": eval_metrics["token_accuracy"],
                "eval/exact_accuracy": eval_metrics["exact_accuracy"],
            }, step=global_step)


def evaluate(model, test_loader, device="cuda"):
    model.eval()
    total_token_correct = 0
    total_tokens = 0
    total_seq_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            batch = {k: v.to(device) for k, v in batch.items()}

            carry = model.initial_carry(batch)

            # 评估时跑满 halt_max_steps
            for _ in range(model.halt_max_steps):
                carry, outputs = model(carry, batch)

            preds = torch.argmax(outputs["logits"], dim=-1)
            labels = batch["labels"]
            mask = (labels != IGNORE_LABEL_ID)

            is_correct = mask & (preds == labels)
            seq_correct = (is_correct.sum(-1) == mask.sum(-1))

            total_token_correct += is_correct.sum().item()
            total_tokens += mask.sum().item()
            total_seq_correct += seq_correct.sum().item()
            total_samples += len(labels)

    return {
        "token_accuracy": total_token_correct / total_tokens,
        "exact_accuracy": total_seq_correct / total_samples,
    }


# ============ Main ============

if __name__ == "__main__":
    # 配置
    DATA_DIR = f"data/sudoku-extreme-${SUBSAMPLE_SIZE}-aug-${NUM_AUG}"
    BATCH_SIZE = 256
    EPOCHS = 100
    LR = 1e-4
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    TEST_SUBSET_SIZE = 1000  # 只取 1000 条做评估

    # ========== W&B 初始化 ==========
    wandb.init(
        project="sudoku-extreme-tiny_trm",
        entity="jarviszhang-new-york-university",
        config={
            "batch_size": BATCH_SIZE,
            "epochs": EPOCHS,
            "lr": LR,
            "weight_decay": 1.0,
            "warmup_steps": 200,
            "hidden_size": 512,
            "num_heads": 8,
            "L_layers": 2,
            "H_cycles": 3,
            "L_cycles": 6,
            "halt_max_steps": 16,
            "test_subset_size": TEST_SUBSET_SIZE,
        }
    )

    # 数据
    train_dataset = PuzzleDataset(DATA_DIR, "train")
    test_dataset_full = PuzzleDataset(DATA_DIR, "test")

    # ========== 从 test 里只取 1000 条 ==========
    np.random.seed(42)
    test_indices = np.random.choice(len(test_dataset_full), size=TEST_SUBSET_SIZE, replace=False)
    test_dataset = Subset(test_dataset_full, test_indices)
    print(f"Using {len(test_dataset)} samples for evaluation (subset of test set)")

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # 模型 (使用之前定义的 TRM)
    model = TRM(
        vocab_size=11,
        seq_len=81,
        hidden_size=512,
        num_heads=8,
        L_layers=2,
        H_cycles=3,
        L_cycles=6,
        halt_max_steps=16,
    )

    print(f"Model params: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")
    print(f"Device: {DEVICE}")

    # W&B watch 模型（可选，记录梯度）
    wandb.watch(model, log="gradients", log_freq=100)

    # 训练
    train(
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        epochs=EPOCHS,
        lr=LR,
        weight_decay=1.0,
        warmup_steps=200,
        eval_interval=10,
        device=DEVICE,
    )

    # 训练结束
    wandb.finish()

epoch,▁▂▄▅▇█
train/epoch_exact_accuracy,▁▁▁▁▁▁
train/epoch_lm_loss,█▅▂▁▁▁
train/epoch_token_accuracy,▁▅▇███
train/exact_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lm_loss,█▇▇▇▇▆▆▆▆▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,▁▁▁▁▂▂▂▂▂▃▄▄▄▄▅▆▆▇▇▇████████████████████
train/q_halt_loss,██████▇▇▆▆▆▆▆▆▆▅▅▅▅▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train/token_accuracy,▁▁▁▁▁▂▂▃▃▃▄▄▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇███████
epoch,6
train/epoch_exact_accuracy,0


Loaded 2000 samples from data/sudoku-extreme-$1000-aug-$1/train
Loaded 422786 samples from data/sudoku-extreme-$1000-aug-$1/test
Using 1000 samples for evaluation (subset of test set)
Model params: 6.83M
Device: cuda


Evaluating: 100%|██████████| 4/4 [00:31<00:00,  7.87s/it]



[Eval] Epoch 10: tok_acc=37.17%, exact_acc=0.00%


Evaluating: 100%|██████████| 4/4 [00:31<00:00,  7.89s/it]



[Eval] Epoch 20: tok_acc=42.75%, exact_acc=0.00%


Epoch 27/100:  75%|███████▌  | 6/8 [00:50<00:16,  8.43s/it, loss=358.9924, tok_acc=44.27%, exact_acc=0.00%, lr=9.99e-05]